In [173]:
import pandas as pd
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [174]:
dfe = pd.read_csv("../Data/Data_cleaned/emotion_cleaned_rudy.csv")[["target", "clean_text"]].dropna()

In [175]:
encoder = LabelEncoder()
dfe["cible"] = encoder.fit_transform(dfe["target"])

## Spliting des données

In [176]:
X_train, X_test, y_train, y_test = train_test_split(dfe["clean_text"], dfe["cible"], train_size=0.8, random_state=1, stratify=dfe["target"])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.5, stratify=y_train)

## Vectorisation

In [177]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer

In [178]:
vectorizer =CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

In [179]:
X_train_t = vectorizer.transform(X_train)
X_test_t = vectorizer.transform(X_test)
X_val_t = vectorizer.transform(X_val)

## Neural Network

In [180]:
model = keras.Sequential()

initializer = keras.initializers.HeNormal()
regularizer = keras.regularizers.L2(0.005)

model.add(keras.layers.Dense(64, input_dim=X_train_t.shape[1], activation="relu",
                             kernel_initializer=initializer,kernel_regularizer=regularizer))
model.add(keras.layers.AlphaDropout(20))
model.add(keras.layers.Dense(32, input_dim=X_train_t.shape[1],kernel_regularizer=regularizer, activation="relu"))
model.add(keras.layers.AlphaDropout(20))
model.add(keras.layers.Dense(6, activation="softmax"))

In [181]:
model.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(),
    optimizer= keras.optimizers.Adam(learning_rate=0.001),
    metrics=["accuracy"]
)

In [183]:
history = model.fit(
    X_train_t.toarray(),
    y_train,
    epochs=100,
    callbacks=keras.callbacks.EarlyStopping(patience=3, monitor="val_loss"),
    batch_size=8,
    validation_data=(X_val_t.toarray(), y_val)
)

Epoch 1/100
1073/1073 [==============================] - 10s 9ms/step - loss: 1.2457 - accuracy: 0.7723 - val_loss: 1.2179 - val_accuracy: 0.7969
Epoch 2/100
1073/1073 [==============================] - 10s 9ms/step - loss: 1.0594 - accuracy: 0.8429 - val_loss: 1.1382 - val_accuracy: 0.8218
Epoch 3/100
1073/1073 [==============================] - 9s 9ms/step - loss: 0.9783 - accuracy: 0.8656 - val_loss: 1.0961 - val_accuracy: 0.8243
Epoch 4/100
1073/1073 [==============================] - 10s 9ms/step - loss: 0.8925 - accuracy: 0.8893 - val_loss: 1.0441 - val_accuracy: 0.8415
Epoch 5/100
1073/1073 [==============================] - 9s 9ms/step - loss: 0.8144 - accuracy: 0.9031 - val_loss: 0.9840 - val_accuracy: 0.8433
Epoch 6/100
1073/1073 [==============================] - 10s 9ms/step - loss: 0.7551 - accuracy: 0.9132 - val_loss: 0.9544 - val_accuracy: 0.8405
Epoch 7/100
1073/1073 [==============================] - 10s 9ms/step - loss: 0.7010 - accuracy: 0.9186 - val_loss: 0.9058 - v

In [184]:
from sklearn.metrics import f1_score
import numpy as np

In [185]:
print(47*'-'+"weighted"+47*'-')
print("Train f1_score:", f1_score(np.argmax(model.predict(X_train_t), axis=1), y_train, average="weighted"))
print("Val f1_score:", f1_score(np.argmax(model.predict(X_val_t), axis=1), y_val, average="weighted"))
print(47*'-'+"macro"+47*'-')
print("Train f1_score:", f1_score(np.argmax(model.predict(X_train_t), axis=1), y_train, average="macro"))
print("Val f1_score:", f1_score(np.argmax(model.predict(X_val_t), axis=1), y_val, average="macro"))
print(47*'-'+"micro"+47*'-')
print("Train f1_score:", f1_score(np.argmax(model.predict(X_train_t), axis=1), y_train, average="micro"))
print("Val f1_score:", f1_score(np.argmax(model.predict(X_val_t), axis=1), y_val, average="micro"))

-----------------------------------------------weighted-----------------------------------------------
Train f1_score: 0.9784136062986091
Val f1_score: 0.850445406869914
-----------------------------------------------macro-----------------------------------------------
Train f1_score: 0.9717728263554425
Val f1_score: 0.8017314411734066
-----------------------------------------------micro-----------------------------------------------
Train f1_score: 0.978326730365882
Val f1_score: 0.8479375436961082


## F1 score weighted par classe

In [205]:
data_score = pd.DataFrame()
data_score["y_pred"] = np.argmax(model.predict(X_val_t),axis=1)
data_score["y_true"] = y_val.reset_index()["cible"]

In [ ]:
def f1_score_classe(classe):
    pass